# C/C++ Library with Python 

The advantage of Python is that it is **flexible and easy** to program. The time it takes to setup a new calulation is therefore short. 

But for certain types of calculations Python (and any other interpreted language) can be **very slow**.

Such calculations may be implemented in a compiled language such as C or Fortran.

In [ ]:
import seuif97
%timeit seuif97.pt2h(15,535)

In [ ]:
from iapws.iapws97 import IAPWS97
%timeit IAPWS97(P=16.10,T=535.10).h

### 1.1 Compiler a shared library with `multiple` files

The shared library with `multiple` code files

* fibonacci.h/c

* funs.h/c

### Once-Only Headers

If a header file happens to be included twice, the compiler will process its contents twice.
This is very likely to cause an error, e.g. when the compiler sees the same structure definition
twice. Even if it does not, it will certainly waste time.

The standard way to prevent this is to enclose the entire real contents of the file in a
conditional, like this:
```c
#ifndef FUNS_H
#define FUNS_H

the entire file

#endif /* !FUNS_H */
```
This construct is commonly known as a wrapper **#ifndef**. When the header is included
again, the conditional will be false, because **FUNS_H** is defined. The preprocessor
will skip over the entire contents of the file, and the compiler will not see it twice.

In [1]:
%%file ./code/gcc/funs.h

#ifndef FUNS_H
#define FUNS_H

double dprod(double *x, int n);
unsigned long LinearFibonacci(int n);

#endif

Writing ./code/gcc/funs.h


In [2]:
%%file ./code/gcc/funs.c

#include <stdio.h>
#include "funs.h"

// x[0]*x[1]*...*x[n-1]
double dprod(double *x, int n)
{
    double y = 1.0;
    for (int i = 0; i < n; i++)
    {
        y *= x[i];
    }
    return y;
}

// The linear time algorithm for finding Fibonacci numbers
//   females(0) =1
//   females(1) = 1
//   females(n + 2) = females(n+1) + females(n）
// n------------                                             
//   f1=  females(n）
//   f2=  females(n+1)
//   fib = f1 + f2;
//   
//    f2=f1; ->females(n+1)
//    f1=fib; ->females(n + 2) 
//n+1----------
//    f1=  females(n+1）
//    f2=  females(n+1+1)
//    fib = f1 + f2;              
//                  
unsigned long LinearFibonacci(int n)
{
  unsigned long fib;
  if (n< 0)
      return -1;
  
  // base case 0 or 1
  if (n == 0||n==1)
      return 1;
 
  unsigned long f1=1,f2 = 1;
  for(int i=2; i<=n; i++)
  {
    // 1 only add
    fib = f1 + f2; 
    // 2 swap  
    f2=f1; 
    f1=fib;
  }    
  return fib;
}

Writing ./code/gcc/funs.c


The **linear time algorithm** for finding `Fibonacci numbers` 

http://www.catonmat.net/blog/on-the-linear-time-algorithm-for-finding-fibonacci-numbers/

#### Building `funs.c`  and `fibonacci.c` into libmultifuns.dll

In [3]:
!gcc -c -O3 -Wall -fPIC  ./code/gcc/funs.c ./code/gcc/fibonacci.c
!gcc -shared -o ./code/gcc/libmultifuns.dll  funs.o  fibonacci.o

In [4]:
!dir .\code\gcc\libmulti*.dll

 驱动器 D 中的卷是 user
 卷的序列号是 DE34-6098

 D:\PySEE\home\notebook\code\gcc 的目录

2018/05/27  19:13            47,646 libmultifuns.dll
               1 个文件         47,646 字节
               0 个目录 22,324,174,848 可用字节


#### Building with makefile

In [5]:
%%file makefile

all: libmultifuns.dll

libmultifuns.dll: multifunsobj
	 gcc  -shared -o ./code/gcc/libmultifuns.dll funs.o fibonacci.o
	 del funs.o fibonacci.o
    
multifunsobj: ./code/gcc/funs.c ./code/gcc/fibonacci.c
	 gcc -c -O3 -Wall -fPIC ./code/gcc/funs.c ./code/gcc/fibonacci.c
     
clean:
	 del .\code\gcc\libmultifuns.dll

Overwriting makefile


In [6]:
!make

gcc -c -O3 -Wall -fPIC ./code/gcc/funs.c ./code/gcc/fibonacci.c
gcc  -shared -o ./code/gcc/libmultifuns.dll funs.o fibonacci.o
del funs.o fibonacci.o


The result is a compiled shared library **`libmultifuns.dll`**

### Using the `variable` in  makefile

A `variable` begins with a **$** and is enclosed within parentheses **(...)** 

In [7]:
%%file makefile

CC=gcc
CFLAGS=-O3 -Wall -fPIC  

INC = -I ./code/gcc/ 

SRCS= ./code/gcc/funs.c ./code/gcc/fibonacci.c 

all: libmultifuns.dll

libmultifuns.dll: multifunsobj
	 $(CC)  -shared -o ./code/gcc/libmultifuns.dll funs.o fibonacci.o
	 del funs.o fibonacci.o
    
multifunsobj: 
	 $(CC) -c $(CFLAGS) $(INC) $(SRCS) 
     
clean:
	 del .\code\gcc\libmultifuns.dll

Overwriting makefile


In [7]:
!make

gcc -c -O3 -Wall -fPIC ./code/gcc/funs.c ./code/gcc/fibonacci.c
gcc  -shared -o ./code/gcc/libmultifuns.dll funs.o fibonacci.o
del funs.o fibonacci.o


### 1.2 ctypes - access the C library

**ctypes** is a foreign function library for Python. It provides C compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

http://docs.python.org/3/library/ctypes.html

We need to load the library and set properties such as the functions return and argument types using the **ctypes** package :

* **ctypes** exports the **cdll**, and on Windows **windll** objects, for loading dynamic link libraries.
   * **cdll.LoadLibrary(name)** : loads the library  which export functions using standard `__cdecl` calling convention
   * **windll.LoadLibrary(name)** : loads the library with `__stdcall` calling convention for the function 


* **argtypes** : the types of the arguments 


* **restype**: the types of return values. 

#### Example `__cdecl` calling convention for the function(windows,Linux)

**msvcrt** is the MS **standard C library** containing most standard C functions, and uses the cdecl calling convention:

In [9]:
from ctypes import *
print(cdll.msvcrt)

<CDLL 'msvcrt', handle 7fff4a030000 at 0x23ab33aed68>


#### Example `__stdcall` calling convention for the function(windows only) 

In [10]:
from ctypes import *
print(windll.kernel32)  

<WinDLL 'kernel32', handle 7fff4a270000 at 0x23ab180ab38>


### Wrap libmultifuns.dll in pure Python.

__cdecl calling convention

In [8]:
%%file ./code/gcc/multifuns.py

from ctypes import cdll,c_void_p,c_int,c_long,c_double,POINTER,byref
import numpy as np


_lib = cdll.LoadLibrary('./code/gcc/libmultifuns.dll')

# double dprod(double *x, int n)
def dprod(x):
    _lib.dprod.argtypes = [np.ctypeslib.ndpointer(dtype=np.float), c_int]
    _lib.dprod.restype  = c_double
    n = len(x)
    x = np.asarray(x, dtype=np.float)
    return _lib.dprod(x, int(n))

#unsigned long fibonacci(int n, unsigned long *fib_cache);
def LinearFibonacci(n):
    _lib.LinearFibonacci.argtypes = [c_int]
    _lib.LinearFibonacci.restype= c_long
    return _lib.LinearFibonacci(int(n))

#unsigned long fibonacci(int n, unsigned long *fib_cache);
def fibonacci(n):
    fibcache = (POINTER(c_long)*n)()
    fib=c_long()
    fib=_lib.LinearFibonacci(int(n),byref(fibcache))
    return  fib

Writing ./code/gcc/multifuns.py


In [9]:
import sys
sys.path.append('./code/gcc')
import multifuns
print(multifuns.LinearFibonacci(10)) 
print(multifuns.fibonacci(10)) 
print(multifuns.dprod([12,3,4,5,6])) 

89
89
4320.0


### Further reading

* The linear time algorithm for finding Fibonacci numbers http://www.catonmat.net/blog/on-the-linear-time-algorithm-for-finding-fibonacci-numbers/


* ctypes

   * http://docs.python.org/3/library/ctypes.html

   * http://scipy-cookbook.readthedocs.io/items/Ctypes.html

   * https://docs.scipy.org/doc/numpy/reference/routines.ctypeslib.html